In [1]:
import os
import cv2 as cv
import numpy as np

dataset_dir = "Basketball-Players-17/test"
base_image_dir = "imgs/court.png"
imgs = os.listdir(dataset_dir)

detector = cv.SIFT_create()
bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)

cv.namedWindow('Feature Matches', cv.WINDOW_NORMAL)
cv.namedWindow('Warped Image', cv.WINDOW_NORMAL)

base_img = cv.imread(base_image_dir)
# base_img = base_img[:,:300]

for img in imgs:
    frame = cv.imread(os.path.join(dataset_dir, img))
    if frame is None:
        continue

    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame = cv.GaussianBlur(frame, (3,3), 10)
    
    kp1, des1 = detector.detectAndCompute(base_img, None)
    kp2, des2 = detector.detectAndCompute(frame, None)

    # Match descriptors
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Draw first 30 matches
    img_matches = cv.drawMatches(base_img, kp1, frame, kp2, matches[:30], None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    if len(matches) > 10:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

            H, mask = cv.findHomography(dst_pts, src_pts, cv.RANSAC, 5.0)
            img2_warped = cv.warpPerspective(frame, H, (base_img.shape[1], base_img.shape[0]))

            cv.imshow('Warped Image', img2_warped)

    cv.imshow('Feature Matches', img_matches)
    key = cv.waitKey(0)
cv.destroyAllWindows()



In [21]:
import os
import cv2 as cv
import numpy as np

dataset_dir = "Basketball-Players-17/test"
base_image_dir = "imgs/rect1-3.png"
imgs = os.listdir(dataset_dir)

cv.namedWindow('img', cv.WINDOW_NORMAL)
cv.namedWindow('frame', cv.WINDOW_NORMAL)

base_img = cv.imread(base_image_dir)

for img in imgs:
    frame = cv.imread(os.path.join(dataset_dir, img))
    if frame is None:
        continue
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    vis = frame.copy()
    edges = cv.Canny(frame, 50, 150)
    lines = cv.HoughLinesP(
        edges,
        rho=1,
        theta=np.pi/180, 
        threshold=40,
        minLineLength=120,
        maxLineGap=15
        )
    
    # Make a copy for visualization
    vis = frame.copy()

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv.line(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # frame = cv.Sobel(frame, 3, 5, 5, )

    cv.imshow('img', vis)
    cv.imshow('frame ', edges)
    key = cv.waitKey(0)
cv.destroyAllWindows()